In [3]:

## FROM DETAILED FIXED ASSET ACCOUNTS TABLES TO ANNUAL FINAL DEMAND ##

# THE FOLLOWING SCRIPT CONTAINS A LIBRARY OF 3 FUNCTIONS WHICH USE MULTI-INDEXED INPUT-OUTPUT DATA TO:

# (A) CONVERT INDUSTRY-SPECIFIC CAPITAL FORMATION SHEETS INTO FULL INDUSTRY ANNUAL DATA FRAMES (AND FIXED ASSETS EXCEL FILES)

# (B) CREATE NEW Z-MATRICES FROM CONCORDANCES TO AGGREGATE AND MATCH THE BENCHMARK COMMODITY FORMAT TO THE FAA COMMODITY FORMAT

# (C) MATCH VALUATION FORMATS OF THE NEW FINAL DEMAND MATRICES AND THE NEW Z-MATRICES (I.E. MATCH FIXED-COSTS)
    ##  For the benchmark tables, see "Concepts and Methods of the U.S. Input-Output Accounts"; PFI does not include depreciation
    ##  For the FAA, in principle should check NIPA

import os
os.chdir(r'C:\Users\albin\Documents\Vienna\Semester 4\Thesis\2) Socio-metabolic branch\1) Capital formation\Jan GitHub repository\Detailed Fixed Asset Accounts')

import pandas as pd
import re

# === Load Excel file ===
file_path = 'Detailnonres\detailnonres_inv1.xlsx'
xls = pd.ExcelFile(file_path)

# === Read the industry mapping from the readme sheet ===
# The industry table starts at row 15 (zero-indexed: skiprows=14)
raw_industry_map = pd.read_excel(
    xls, 
    sheet_name='readme', 
    skiprows=14, 
    usecols='A:B', 
    names=['Industry Name', 'BEA Code']
)

# === Helper to validate BEA code ===
def is_valid_bea_code(x):
    try:
        return bool(re.fullmatch(r'\d{3,4}[A-Z]*', x)) and x != '--------'
    except:
        return False

# === Strip whitespace and apostrophes before filtering ===
raw_industry_map['BEA Code'] = raw_industry_map['BEA Code'].astype(str).str.strip().str.lstrip("'")
raw_industry_map['Industry Name'] = raw_industry_map['Industry Name'].astype(str).str.strip()

# === Filter and clean valid industry entries ===
mask = raw_industry_map['BEA Code'].apply(is_valid_bea_code)
valid_industries = raw_industry_map[mask].copy()

# === Create MultiIndex: (Industry Name, Code) ===
industry_index = pd.MultiIndex.from_frame(
    valid_industries[['Industry Name', 'BEA Code']],
    names=['Industry Name', 'BEA Code']
)

# === Loop over years and collect data ===
all_year_dfs = {}

# Define years of interest
years = list(range(1901, 1902))

# === Loop over years and collect data ===
all_year_dfs = {}

# Define years of interest
years = list(range(1901, 2024))

for year in years:
    print(f"Processing year: {year}")
    year_data = []

    for industry_name, bea_code in industry_index:
        try:
            df = pd.read_excel(
                xls, 
                sheet_name=bea_code, 
                skiprows=5  # Header is row 6 (0-based index = 5)
            )

            # --- Clean column names ---
            df.columns = df.columns.astype(str).str.strip()

            if str(year) not in df.columns:
                continue  # Skip if year column not present

            # --- Extract relevant columns ---
            temp = df[['Asset Codes', 'NIPA Asset Types', str(year)]].copy()
            temp.columns = ['Asset Code', 'NIPA Asset Type', f'{bea_code}']

            # === Clean and filter asset rows ===
            temp['Asset Code'] = temp['Asset Code'].astype(str).str.strip().str.lstrip("'")
            temp['NIPA Asset Type'] = temp['NIPA Asset Type'].astype(str).str.strip()

            # Remove aggregate rows where asset name contains "total" (case-insensitive) and remove empty top column
            temp = temp[~temp['NIPA Asset Type'].str.lower().str.contains('total')].iloc[1:]

            # --- Set MultiIndex on asset code and asset name ---
            temp.set_index(['Asset Code', 'NIPA Asset Type'], inplace=True)

            # --- Rename column to industry ---
            year_data.append(temp.rename(columns={f'{bea_code}': (industry_name, bea_code)}))

        except Exception as e:
            print(f"Failed to process {bea_code} for year {year}: {e}")
            continue

    # === Combine across all industries for this year ===
    if year_data:
        disGFCF_year = pd.concat(year_data, axis=1)
        disGFCF_year.columns = pd.MultiIndex.from_tuples(disGFCF_year.columns, names=['Industry Name', 'BEA Code'])
        all_year_dfs[year] = disGFCF_year

        ## Save to excel ##
        ## disGFCF_year.to_excel(f"New Final demand tables/disGFCF_{year}.xlsx")
        
        print(f"Finished year {year}, shape: {disGFCF_year.shape}")
    else:
        print(f"No data found for year {year}")



Processing year: 1901
Finished year 1901, shape: (96, 74)
Processing year: 1902
Finished year 1902, shape: (96, 74)
Processing year: 1903
Finished year 1903, shape: (96, 74)
Processing year: 1904
Finished year 1904, shape: (96, 74)
Processing year: 1905
Finished year 1905, shape: (96, 74)
Processing year: 1906
Finished year 1906, shape: (96, 74)
Processing year: 1907
Finished year 1907, shape: (96, 74)
Processing year: 1908
Finished year 1908, shape: (96, 74)
Processing year: 1909
Finished year 1909, shape: (96, 74)
Processing year: 1910
Finished year 1910, shape: (96, 74)
Processing year: 1911
Finished year 1911, shape: (96, 74)
Processing year: 1912
Finished year 1912, shape: (96, 74)
Processing year: 1913
Finished year 1913, shape: (96, 74)
Processing year: 1914
Finished year 1914, shape: (96, 74)
Processing year: 1915
Finished year 1915, shape: (96, 74)
Processing year: 1916
Finished year 1916, shape: (96, 74)
Processing year: 1917
Finished year 1917, shape: (96, 74)
Processing yea

In [12]:
all_year_dfs[1963].to_excel("New Final demand tables\disGFCF_1963.xlsx")

print(valid_industries['BEA Code'].unique())

['110C' '113F' '2110' '2120' '2130' '2211' '2212' '2213' '2300' '3210'
 '3270' '3310' '3320' '3330' '3340' '3350' '336M' '336O' '3370' '3380'
 '3390' '3110' '3120' '313T' '315A' '3220' '3230' '3240' '3250' '3260'
 '4210' '4220' '4410' '4450' '4520' '442R' '4810' '4820' '4830' '4840'
 '4850' '4860' '487S' '4930' '5110' '5120' '5130' '5140' '5210' '5221'
 '5223' '5229' '5230' '524A' '5243' '5250' '5310' '5320' '5411' '5415'
 '5412' '5500' '5610' '5620' '6100' '6210' '622H' '6230' '6240' '711A'
 '7130' '7210' '7220' '8100']
